In [ ]:
import pandas as pd
import re
from konlpy.tag import Mecab
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from tqdm import tqdm  # 반복문 진행상황 보기
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
import numpy as np
from gensim.models.callbacks import CallbackAny2Vec
import time

In [ ]:
import pymysql
import pandas as pd
host_name = 'localhost'
host_port = 3306
username='root'
password='12345678'
database_name='news_team_4'
db = pymysql.connect(
    host=host_name,
    port=host_port,
    user=username,
    password=password,
    db=database_name,
    charset='utf8'
)
SQL='SELECT * FROM tb_news_team_4'
df=pd.read_sql(SQL, db)

In [ ]:
df = df[df['Content'] != '']

In [ ]:
df['MainCategory'].unique()

In [ ]:
df.loc[df['MainCategory'] == 'entertain', 'MainCategory'] = '연예'



In [ ]:
df.loc[df['MainCategory'] == '승부예측', 'MainCategory'] = '스포츠'


In [ ]:
df = df[df.MainCategory != '영상']


In [ ]:
df.loc[df['MainCategory'] == '문화', 'MainCategory'] = '생활/문화'

In [ ]:
df.loc[df['MainCategory'] == 'IT', 'MainCategory'] = 'IT/과학'

In [ ]:
df.loc[df['MainCategory'] == '세계', 'MainCategory'] = '국제'

In [ ]:
sports = df[df['MainCategory'] == '스포츠']
enter = df[df['MainCategory'] == '연예']
social = df[df['MainCategory'] == '사회']
culture = df[df['MainCategory'] == '생활/문화']
politics = df[df['MainCategory'] == '정치']
economy = df[df['MainCategory'] == '경제']
inter = df[df['MainCategory'] == '국제']
it_sci = df[df['MainCategory'] == 'IT/과학']

In [ ]:
s_sports =sports.sample(frac=0.2, replace=False, ignore_index=True, axis=0)
s_enter = enter.sample(frac=0.2, replace=False, ignore_index=True, axis=0)
s_social = social.sample(frac=0.2, replace=False, ignore_index=True, axis=0)
s_culture = culture.sample(frac=0.2, replace=False, ignore_index=True, axis=0)
s_politics = politics.sample(frac=0.2, replace=False, ignore_index=True, axis=0)
s_economy = economy.sample(frac=0.2, replace=False, ignore_index=True, axis=0)
s_inter = inter.sample(frac=0.2, replace=False, ignore_index=True, axis=0)
s_it_sci = it_sci.sample(frac=0.2, replace=False, ignore_index=True, axis=0)

In [ ]:
df_list = [s_sports, s_enter, s_social, s_culture, s_politics, s_economy, s_inter, s_it_sci]

In [ ]:
df_all = pd.read_csv('/Users/stillssi/Desktop/NEWS_TEAM_4/MDL/testmodel/sample_df.csv')

In [ ]:
df_all = pd.concat(df_list, ignore_index=True)
df_all = df_all.drop_duplicates(['URL'])
df_all = df_all[df_all['Content'] != '카톡']
df_all = df_all[df_all['Content'] != '']
df_all.to_csv('./sample_df.csv')

In [ ]:
len(df_all)

In [ ]:
tagged_corpus_list = []
for i in range(len(df_all)):
    content = df_all['Content'].iloc[i]
    
    tag = df_all['URL'].iloc[i]
    tagged_corpus_list.append(TaggedDocument(tags=[tag], words=content))

In [ ]:
len(tagged_corpus_list)

In [ ]:
model = Doc2Vec(vector_size=100, alpha=0.025, min_alpha=0.01, workers=8)
model.build_vocab(tagged_corpus_list)
model.train(tqdm(tagged_corpus_list, total=len(tagged_corpus_list)), total_examples=model.corpus_count, epochs=10)

In [ ]:

model.save('trainedMDL')
loaded_model = Doc2Vec.load('trainedMDL')